rough mayorga

In [1]:
import math
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from datetime import datetime

from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
import sklearn.preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.feature_selection import SelectKBest, f_regression, RFE

In [2]:
df = pd.read_csv('texas.csv')

In [3]:
from Prepare import prepare_tex

In [4]:
df = prepare_tex(df)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144733 entries, 0 to 144737
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   agency_id       144733 non-null  int64         
 1   agency          144733 non-null  object        
 2   lastname        144733 non-null  object        
 3   firstname       144733 non-null  object        
 4   title           144733 non-null  object        
 5   race            144733 non-null  object        
 6   sex             144733 non-null  object        
 7   emptype         144733 non-null  object        
 8   hire_date       144733 non-null  datetime64[ns]
 9   hours_worked    144733 non-null  float64       
 10  monthly_salary  144733 non-null  float64       
 11  annual_salary   144733 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(7)
memory usage: 14.4+ MB


In [6]:
df.head()

,agency_id,agency,lastname,firstname,title,race,sex,emptype,hire_date,hours_worked,monthly_salary,annual_salary
0,101,SENATE ...,GILLIAM,STACEY,LEG. OFFICIAL/ADMINISTRATOR ...,WHITE,FEMALE,URP - UNCLASSIFIED REGULAR PART-TIME,2013-07-01,20.0,8100.0,97200.0
1,104,LEGISLATIVE BUDGET BOARD ...,GILLIAM,STACEY,COMMITTEE DIRECTOR ...,WHITE,FEMALE,URP - UNCLASSIFIED REGULAR PART-TIME,2017-11-01,20.0,7000.0,84000.0
2,101,SENATE ...,NELSON,DAVID,LEG. OFFICIAL/ADMINISTRATOR ...,WHITE,MALE,URP - UNCLASSIFIED REGULAR PART-TIME,1998-12-01,20.0,9500.0,114000.0
3,104,LEGISLATIVE BUDGET BOARD ...,NELSON,DAVID,SENIOR BUDGET ADVISOR ...,WHITE,MALE,URP - UNCLASSIFIED REGULAR PART-TIME,2017-11-01,20.0,8000.0,96000.0
4,101,SENATE ...,ROCHA,MARIE,LEG. SERVICE/MAINTENANCE ...,HISPANIC,FEMALE,URF - UNCLASSIFIED REGULAR FULL-TIME,2003-05-01,41.0,3365.4,40384.8
